In [ ]:
import jupyter_black

jupyter_black.load()

import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_predict
import matplotlib.pyplot as plt
from pmdarima import auto_arima
import os

## Setting up the file path and forecast time horizon

In [ ]:
file_path = "/Users/66789/Documents/GitHub/aag-bain-f2fw/"
os.chdir(file_path)
datafile = "data/230428 trucking2303_shared with AAG_mod.xlsx"

forecast_start = pd.datetime(2023, 3, 1)
forecast_end = pd.datetime(2025, 12, 1)

### Reading the data to train the model
* Considers both monthly and quarterly data

In [ ]:
def read_data(freq: str):
    df = pd.read_excel(datafile, sheet_name=freq).T.reset_index(drop=True)
    new_header = df.iloc[1]  # grab the first row for the header
    df = df[3:]  # take the data less the header row
    df.columns = new_header  # set the header row as the df header
    df.drop(["Label: (SA = seasonally adjusted)"], axis=1, inplace=True)
    df = df.loc[:, df.columns.notna()]
    df = df.loc[:, ~df.columns.str.startswith("TCI Component")]
    df = df.loc[:, ~df.columns.str.startswith("Unnamed")]

    df.dropna(axis=0, how="all", inplace=True)
    df = df.apply(pd.to_numeric, downcast="float", errors="coerce")

    if freq == "QUARTERLY":
        df["Date"] = pd.date_range("2000-01-01", "2025-12-01", freq="QS")
        # df["Year"] = df["Date"].dt.year
        # df["Quarter"] = df["Date"].dt.quarter
    if freq == "MONTHLY":
        df["Date"] = pd.date_range("2000-01-01", "2025-01-01", freq="MS")
        # df["Year"] = df["Date"].dt.year
        # df["Month"] = df["Date"].dt.month

    # df.dropna(inplace=True)
    df.set_index("Date", inplace=True)
    return df

In [ ]:
df_quarterly = read_data("QUARTERLY")
df_monthly = read_data("MONTHLY")

spot_col = "Total TL: Spot Rate (exc. FSC, SA)"
contract_col = "Total TL: Contract Rate (exc. FSC, SA)"

### Reads the driver values for the forecast scenario

In [ ]:
## Reading the file with scenario driver values
df_scenario = pd.read_excel(datafile, sheet_name="QUARTERLY (Scenario1)").T.reset_index(
    drop=True
)

## Cleaning the file
new_header = df_scenario.iloc[1]  # grab the first row for the header
df_scenario = df_scenario[3:]  # take the data less the header row
df_scenario.columns = new_header  # set the header row as the df header
# Removing unused columns
df_scenario.drop(["Label: (SA = seasonally adjusted)"], axis=1, inplace=True)
df_scenario = df_scenario.loc[:, df_scenario.columns.notna()]
df_scenario = df_scenario.loc[:, ~df_scenario.columns.str.startswith("TCI Component")]
df_scenario = df_scenario.loc[:, ~df_scenario.columns.str.startswith("Unnamed")]
df_scenario.dropna(axis=0, how="all", inplace=True)  # Removing empty rows
# Setting all values to numbers
df_scenario = df_scenario.apply(pd.to_numeric, downcast="float", errors="coerce")
# Index rows by dates
df_scenario["Date"] = pd.date_range(forecast_start, forecast_end, freq="QS")
df_scenario.set_index("Date", inplace=True)

df_scenario.head()

## Training the model and scenario analysis
* The forecasted values (and their 95% confidence intervals) will be outputted in CSV files in the file path

In [ ]:
train_start = pd.datetime(2008, 1, 1)
train_end = pd.datetime(2025, 12, 30)


df = df_quarterly.copy()
test_end = pd.datetime(2025, 12, 30)

for target_col in [contract_col, spot_col]:
    if target_col == spot_col:
        exog = [
            "Active Truck Utilization (SA)",
            "Total Truck Loadings (SA)",
            "National Avg. Diesel Fuel Price ($/Gal.)",
            "Driver Labor Index (1992=100, SA)",
            "3 Month T-Bill Rate, %",
        ]
    elif target_col == contract_col:
        exog = [
            "Active Truck Utilization (SA)",
            "Total Truck Loadings (SA)",
            "National Avg. Diesel Fuel Price ($/Gal.)",
            # "Driver Labor Index (1992=100, SA)",
            # "3 Month T-Bill Rate, %",
        ]

    train_df = df[(df.index >= train_start) & (df.index <= train_end)]
    forecast_df = df_scenario[
        (df_scenario.index >= train_start) & (df_scenario.index <= forecast_end)
    ]

    best_model = auto_arima(
        train_df[target_col],
        start_p=0,
        start_q=0,
        X=train_df[exog],
        test="adf",  # use adftest to find optimal 'd'
        max_p=3,
        max_q=3,  # maximum p and q
        m=1,  # frequency of series
        d=None,  # let model determine 'd'
        seasonal=True,
        start_P=0,
        D=0,
        trace=True,
        error_action="ignore",
        suppress_warnings=True,
        stepwise=True,
    )

    # Build Model
    model = ARIMA(
        train_df[target_col],
        exog=train_df[exog],
        order=best_model.order,
    )
    fitted = model.fit()
    print(fitted.summary())

    # Plot diagnostics
    fitted.plot_diagnostics(figsize=(16, 9))
    plt.show()

    # Forecast
    pred = fitted.get_prediction(
        start=forecast_df.index[0],
        end=forecast_df.index[-1],
        exog=forecast_df[exog],
        dynamic=False,
    )

    print(model.order)
    print(train_df[exog].dropna(axis=1).columns)
    pd.concat([pred.predicted_mean, pred.conf_int(alpha=0.05)], axis=1).to_csv(
        "Scenario_Forecast_" + target_col + ".csv", index=False
    )

    # Plot
    fig, ax = plt.subplots()
    ax = df[target_col].dropna().plot(ax=ax)

    plot_predict(
        fitted,
        forecast_df.index[1],
        forecast_df.index[-1],
        ax=ax,
        exog=forecast_df[exog].dropna(axis=1),
    )

    plt.xlim([train_start, forecast_end])

    plt.show()